In [1]:
import pyro
import torch
import pyro.distributions as dist 
import numpy as np
from matplotlib import pyplot as plt

In [10]:
data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))

In [13]:
alpha0 = torch.tensor(10.0)
beta0 = torch.tensor(10.0)

In [16]:
def model(data):
    #sample f from the beta prior
    f = pyro.sample('latent_fairness', dist.Beta(alpha0, beta0))
    #loop over the observed data using pyro.sample with the obs keyword
    for i in range(len(data)):
        pyro.sample(f'obs_{i}', dist.Bernoulli(f), obs=data[i])

In [62]:
def model(data):
    #sample f from the beta prior
    f = pyro.sample('latent_fairness', dist.Beta(alpha0, beta0))
    #loop over the observed data using pyro.sample with the obs keyword
    for i in pyro.plate('data_loop', len(data)):
        pyro.sample(f'obs_{i}', dist.Bernoulli(f), obs=data[i])

In [63]:
#DO NOT do _this_
my_reified_list = list(pyro.plate('data_loop', len(data)))
for i in my_reified_list:
    pyro.sample(f'obs_{i}', dist.Bernoulli(f), obs=data[i])

In [76]:
data = torch.zeros(10)
data[0:6] = torch.ones(6)
data

tensor([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [79]:
%%timeit 

with pyro.plate('observed_data'):
    pyro.sample('obs', dist.Bernoulli(f), obs=data)

37.5 µs ± 240 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [81]:
%timeit model(data)

480 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [121]:
for i in pyro.plate('data_loop', len(data), subsample_size=5):
    pyro.sample(f'obs_{i}', dist.Bernoulli(f), obs=data[i])

In [142]:
with pyro.plate('observed_data', size=10, subsample_size=5) as ind:
    pyro.sample('obs', dist.Bernoulli(f), obs=data.index_select(0, ind))